# Dropping 3 target column and replacing with 1 target column. Performing feature selection and training with random forest.

In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import warnings

df = pd.read_csv("2024Nov_HT_Phase2.4.csv", encoding='latin-1')


In [2]:
import pandas as pd

# Subset the target columns
target_columns = ['aphp', 'nhph', 'secondary_hp']
target_data = df[target_columns]

# Find unique combinations and their frequencies
combinations = target_data.value_counts().reset_index(name='Frequency')

# Sort by Frequency in descending order
combinations = combinations.sort_values(by='Frequency', ascending=False)

# Print the result
print(combinations)


   aphp  nhph  secondary_hp  Frequency
0     0     0             0        749
1     0     1             0        239
2     1     0             0        232
3     0     0             1        150


In [3]:
# Check for non-numeric columns in the entire dataset
print(df.dtypes)

# Check for missing or non-numeric values in the dataset
print(df.isnull().sum())  # To check for missing values

# Check for any string values in the entire dataset
print(df[df.map(lambda x: isinstance(x, str))])  # To check for string values


aphp                                int64
nhph                                int64
tertiary_hp                         int64
secondary_hp                        int64
age                                 int64
                                    ...  
alp_combined_with_cl_po4           object
third_surgery_pathology_report     object
fourth_surgery_pathology_report     int64
triglycerides                       int64
hdl_cholesterol                     int64
Length: 73, dtype: object
aphp                               0
nhph                               0
tertiary_hp                        0
secondary_hp                       0
age                                0
                                  ..
alp_combined_with_cl_po4           0
third_surgery_pathology_report     0
fourth_surgery_pathology_report    0
triglycerides                      0
hdl_cholesterol                    0
Length: 73, dtype: int64
      aphp  nhph  tertiary_hp  secondary_hp  age  gender  height  weight  bmi  \

In [4]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel

# Assuming 'df' is your DataFrame
target_columns = ['aphp', 'nhph', 'secondary_hp']  # Columns 1, 2, and 4

# Map combinations of target columns to a single target value
combination_to_target = {
    (0, 0, 0): 0,
    (0, 1, 0): 1,
    (1, 0, 0): 2,
    (0, 0, 1): 3
}

# Create a new target column by applying the mapping to the relevant columns
df['new_target'] = df[target_columns].apply(lambda row: combination_to_target[tuple(row)], axis=1)

# Drop the original target columns
df = df.drop(columns=target_columns)

# Split the dataset into features (X) and target (y)
X = df.drop(columns=['new_target'])  # Features (all columns except the new target)
y = df['new_target']  # Target (the new target column)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)

# Feature selection using the trained model
sfm = SelectFromModel(rf, threshold="mean", max_features=10)
sfm.fit(X_train, y_train)

# Get the selected features
selected_features = X.columns[sfm.get_support()]
print(f"Selected Features: {selected_features}")


ValueError: could not convert string to float: ' '